In [1]:
import pandas as pd
import json
from module.load_json import *
from module.utils import *
from module.maps import *

In [2]:
train_path = 'dataset/NIKL_ABSA_2021_v1.0/EXSA2112203180.json'
dev_path = 'dataset/NIKL_ABSA_2021_v1.0/EXSA2122203180.json'

In [3]:
train = jsonload(train_path)
dev = jsonload(dev_path)
train = pd.DataFrame(train['document'])
dev = pd.DataFrame(dev['document'])
total = pd.concat([train, dev]).reset_index(drop=True)

In [4]:
total.domain.unique()
categories = ['제품 기타', '전자기기', '화장품/세정제']

checker = total.domain.apply(lambda x: x in categories)
total = total[checker]
total = total[total.isna().any(axis=1) == False]
total = total[['sentence', 'opinions']].reset_index(drop=True)

In [5]:
def collect_sentences(sentences):
    collected_sentences = []
    for sentence in sentences:
        collected_sentences.append(sentence['sentence_form'])
    return ' '.join(collected_sentences)

In [6]:
def collect_opinions(opinions):
    collected_opinions = []
    for opinion in opinions:
        category = opinion['category']
        polarity = opinion['opinion polarity']
        if polarity != 'conflict':
            collected_opinions.append([category, [], polarity])
    return collected_opinions

In [7]:
total['sentence'] = total.sentence.apply(collect_sentences)
total['opinions'] = total.opinions.apply(collect_opinions)
total = total[total.opinions.apply(lambda x: x == []) == False]
total = total.reset_index()
total = total.set_axis(['id', 'sentence_form', 'annotation'], axis='columns')

In [8]:
total

,id,sentence_form,annotation
0,0,잔차 사서 일주일쯤 지났다. 그동안 비가 와서 생각만큼 많이 타진 못했고 그래도 며...,"[[본품#일반, [], neutral]]"
1,2,이런 젠장.. 동영상 재생하면서 자막 중 모르는 내용 있으면 터치해서 바로 검색하는...,"[[제품 전체#가격, [], negative]]"
2,3,스맛폰 열풍이 분 지가 언제인데 이제서야 미라크로 막차 탐. 내 예상대로 우리집에서...,"[[본품#일반, [], negative]]"
3,4,나시라 읽고 슬리브리스라고 쓴다. -_-; 지식인이므로.. ㅋㅋㅋㅋ 예전에는 혼자서...,"[[본품#품질, [], neutral], [제품 전체#가격, [], neutral]..."
4,5,화장품 가게에 갔다가 샘플로 받은 풋 마스크. 발 전용 팩이란다. -_-; 얼굴에도...,"[[본품#품질, [], neutral]]"
...,...,...,...
619,771,미니멀 여파인지 하고싶은 머리도 딱히 없고 짧은 머리는 또 지긋해서 심심한데 붙임머...,"[[본품#편의성, [], positive]]"
620,772,생긴건 바이오더마같이 생겼는데 가격은 절반이네 캬 플란테시스템 이름이 쓸데없이 어렵...,"[[본품#일반, [], positive]]"
621,773,http://www.head-fi.org/t/705687/review-of-audi...,"[[제품 전체#가격, [], neutral]]"
622,774,이마트에서9900원 ㅠㅠ 비싸 ㅠㅠㅠ,"[[제품 전체#가격, [], negative]]"


In [9]:
count_tags(total, entity_property_pair)

tags found:  1195
tag set of df:  20
tag set of offered:  25
difference:  {'본품#인지도', '본품#가격', '브랜드#디자인', '제품 전체#다양성', '패키지/구성품#가격'}
제품 전체#일반		290
본품#품질		178
본품#일반		148
제품 전체#품질		103
제품 전체#디자인	75
본품#편의성		58
제품 전체#인지도	58
제품 전체#편의성	56
브랜드#일반		48
패키지/구성품#디자인	43
제품 전체#가격		33
패키지/구성품#편의성	33
패키지/구성품#일반	19
본품#다양성		19
본품#디자인		9
패키지/구성품#품질	7
브랜드#품질		7
브랜드#인지도		7
브랜드#가격		3
패키지/구성품#다양성	1


In [10]:
DATA_V = 'uncleaned_v23'
save_path = f'./dataset/{DATA_V}'
print(save_path)

./dataset/uncleaned_v23


In [11]:
!mkdir -p {save_path}

total.to_json(f'{save_path}/absa2021.json', force_ascii=False)